In [2]:
# Import packages
import os
import warnings
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from skimage import io
from skimage.measure import regionprops_table
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=Warning)

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42


def intensity_median(regionmask, intensity):
    return np.median(intensity[regionmask])


dir_output = (
    "/cluster/project/treutlein/DATA/imaging/4i_Data/Brain_ECM_4i/Brain_ECM_4i_dapi/"
)

dir_bg_subtracted = Path(dir_output, "bg_subtracted")
dir_segmented = Path(dir_output, "segmented")

table_data = {
    "3": {"Range": "227-237", "Condition": "Matrigel", "Day": "11"},
    "4.1": {"Range": "238-245", "Condition": "No Matrix", "Day": "11"},
    "4.2": {"Range": "246-251", "Condition": "No Matrix", "Day": "11"},
    "4.3": {"Range": "252-265", "Condition": "No Matrix", "Day": "11"},
    "4.4": {"Range": "266-271", "Condition": "No Matrix", "Day": "7"},
    "5.1": {"Range": "272-282", "Condition": "Matrigel", "Day": "7"},
    "5.2": {"Range": "283-289", "Condition": "Matrigel", "Day": "7"},
    "5.3": {"Range": "290-296", "Condition": "Matrigel", "Day": "7"},
    "5.4": {"Range": "297-303", "Condition": "Matrigel", "Day": "7"},
    "6.1": {"Range": "304-308", "Condition": "No Matrix", "Day": "7"},
    "6.2": {"Range": "309-317", "Condition": "No Matrix", "Day": "7"},
    "6.3": {"Range": "318-326", "Condition": "No Matrix", "Day": "7"},
    "6.4": {"Range": "327-334", "Condition": "No Matrix", "Day": "7"},
    "7.1": {"Range": "335-345", "Condition": "Matrigel", "Day": "16"},
    "7.2": {"Range": "346-356", "Condition": "Matrigel", "Day": "16"},
    "7.3": {"Range": "357-360", "Condition": "Matrigel", "Day": "16"},
    "7.4": {"Range": "360-364", "Condition": "Matrigel", "Day": "16"},
    "8": {"Range": "365-371", "Condition": "No Matrix", "Day": "16"},
}

channel = "02"
cycle = "26"
samples = [
    "R294_0",
    "R277_0",
    "R300_0",
    "R284_0",
    "R315_0",
    "R268_0",
    "R304_0",
    "R232_0",
    "R249_0",
    "R253_0",
    "R347_0",
    "R347_1",
    "R338_0",
    "R338_1",
    "R365_0",
    "R365_1",
    "R365_2",
    "R365_3",
]

new_props = pd.DataFrame()

for sample in tqdm(samples):
    img_label = io.imread(Path(dir_segmented, sample + ".tif"))
    sample_split = sample.split("_")[0]
    img_name = Path(
        dir_bg_subtracted, f"{sample}/Stitched_C{cycle}_{sample_split}_ch{channel}.tif"
    )
    imgs = io.imread(img_name)

    region_props = pd.DataFrame(
        regionprops_table(
            label_image=img_label,
            intensity_image=imgs,
            properties=(
                "label",
                "centroid",
                "area",
                "intensity_mean",
                "intensity_max",
                "intensity_min",
            ),
            extra_properties=(intensity_median,),
        )
    )

    new_prop = region_props
    new_prop["sample"] = sample
    new_prop["sample_num"] = (
        new_prop["sample"]
        .str.replace("R", "", regex=False)
        .str.replace(r"_\d+", "", regex=True)
    )
    new_prop["sample_num"] = new_prop["sample_num"].astype(int)

    new_props = pd.concat([new_props, new_prop])

new_props["Block"] = np.nan
new_props["Condition"] = np.nan
new_props["Day"] = np.nan

for block in table_data:
    ranges = table_data[block]["Range"].split("-")
    range_pd = new_props["sample_num"].between(int(ranges[0]), int(ranges[1]))
    new_props.loc[range_pd, "Day"] = table_data[block]["Day"]
    new_props.loc[range_pd, "Block"] = block
    new_props.loc[range_pd, "Condition"] = table_data[block]["Condition"]

color_pallete_perturbation = {"Matrigel": "#17ad97", "No Matrix": "#4d4d4d"}

# YAP1 time course plot
yap_time_course = pd.DataFrame()
yap_time_course["Log median expression"] = np.log1p(new_props["intensity_median"])
yap_time_course["Condition"] = new_props["Condition"]
yap_time_course["Day"] = new_props["Day"]
yap_time_course["protein"] = "YAP1"
yap_time_course.to_csv("anndata/yap1_time_course_median.csv")

fig, ax = plt.subplots(figsize=(4, 3))
sns.despine(left=False, bottom=False, right=True)

sns.violinplot(
    yap_time_course,
    x="Day",
    y="Log median expression",
    split=True,
    palette=color_pallete_perturbation,
    gap=0.08,
    hue="Condition",
    inner=None,
    cut=0,
).legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=16)
fig.savefig(
    "figures/YAP1_median_time_course_extended_data_Fig_12a.pdf", bbox_inches="tight"
)
plt.close()

# YAP1 day 16 plot
yap_day_16_course = pd.DataFrame()
yap_day_16_course["Log expression \n (mean)"] = np.log1p(new_props["intensity_mean"])
yap_day_16_course["Condition"] = new_props["Condition"]
yap_day_16_course["Day"] = new_props["Day"]
yap_day_16_course["protein"] = "YAP1"
yap_day_16_course = yap_day_16_course[yap_day_16_course["Day"] == "16"]
yap_day_16_course.to_csv("anndata/yap1_day_16_mean.csv")

fig, ax = plt.subplots(figsize=(1, 2))
sns.despine(left=False, bottom=False, right=True)

sns.violinplot(
    yap_day_16_course,
    x="Day",
    y="Log expression \n (mean)",
    split=True,
    palette=color_pallete_perturbation,
    gap=0.05,
    hue="Condition",
    cut=0,
).legend(loc="center left", bbox_to_anchor=(1.0, 0.5), fontsize=16)
ax.set(ylim=(0, 8))
fig.savefig("figures/YAP1_mean_expression_Fig_5b.pdf", bbox_inches="tight")
plt.close()

100%|██████████| 18/18 [00:05<00:00,  3.30it/s]
